# FLoRA implementation

In [8]:
import sys
sys.path.append("../../")

import glob
import os
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from collections import defaultdict
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF

from constants import DATASETS, SKEWS, NR_PARTIES, HP_GRID
from clustering import get_gt_data, get_client_data, closest_gt_to_point
from read_data import get_federated_res, get_federated_val_acc

DATA_PATH = "experiments/extended_data"

In [4]:
def get_derived_params(prediction, dataset_file_name, fedavg_acc):
  derived_params = list()

  for pred in prediction:
    d_lr, d_mom, d_bs = pred[1][0], pred[1][1], pred[1][2]
    (c_lr, c_mom, c_bs, c_acc), min_diff = closest_gt_to_point([d_lr, d_mom, d_bs], dataset_file_name)
    (c_lrH, c_momH, c_bsH, c_accH), min_diffH = closest_gt_to_point([d_lr, d_mom, d_bs/2], dataset_file_name)

    derived_params.append({
        "lr_mom_bs": (d_lr, d_mom, int(d_bs)),
        "closest_lr_mom_bs_acc": (c_lr, c_mom, c_bs, c_acc),
        "diff_acc": fedavg_acc - c_acc,
        "diff_space": min_diff,

        "heurBS": {
            "lr_mom_bs_HEURBS": (d_lr,d_mom, int(d_bs)/2),
            "closest_lr_mom_bs_acc_HEURBS": (c_lrH, c_momH, c_bsH, c_accH),
            "diff_acc_HEURBS": fedavg_acc - c_accH,
            "diff_space_HEURBS": min_diffH,
        }
    }) 

  return derived_params

def get_lr_mom_bs_range(dataset_name, df):
  server_data = get_gt_data(dataset_name)

  lr_range = server_data.server_lr.unique()
  mom_range = server_data.server_mom.unique()
  bs_range = server_data.batch_size.unique()

  lr_range_cl = df.client_lr.unique()
  mom_range_cl = df.client_mom.unique()
  bs_range_cl = df.batch_size.unique()

  return lr_range_cl, mom_range_cl, bs_range_cl

def get_prediction(df, predicted_losses, in3D=False):
  prediction = []
  for pred in predicted_losses:
    batch_size = 16
    if in3D:
      batch_size = pred[1][2]
    else:
      batch_size = df.loc[(df['client_lr'] == pred[1][0]) & (df['client_mom'] == pred[1][1])]['batch_size'].mode()[0]
    prediction.append((pred[0], (pred[1][0], pred[1][1], batch_size)))
  
  return prediction


In [5]:
def predict_best_HPs_SGM(dataset_name, withUncertainty=False, in3D=False, alpha=1e-1):
  client_data = get_client_data(dataset_name)
  samples_per_client = client_data[client_data['clientID']==1].shape[0]
  clientIDs = np.unique(client_data['clientID'].tolist())

  df = pd.DataFrame()
  for client_id in clientIDs:
    tmp = client_data.loc[client_data.clientID == client_id]
    tmp = tmp[['client_lr', 'client_mom', 'batch_size', 'accuracy', 'loss']]
    df = pd.concat([df, tmp])

  X, y = df[['client_lr', 'client_mom']].values, df[['loss']].values
  if in3D:
    X, y = df[['client_lr', 'client_mom', 'batch_size']].values, df[['loss']].values

  if withUncertainty:
    kernel = ConstantKernel(1.0, (1e-7, 1e7)) * RBF(10.0, (1e-7, 1e7))
    regr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=0, random_state=12)
    regr.fit(X,y)
  else:
    regr = RandomForestRegressor(random_state=12)
    regr.fit(X, y.ravel())

  lr_range, mom_range, bs_range = get_lr_mom_bs_range(dataset_name, df)

  predicted_losses = []
  for lr in lr_range:
    for mom in mom_range:
      if in3D:
        for bs in bs_range:
          if withUncertainty:
            mean, std = regr.predict([[lr, mom, bs]], return_std=True)
            predicted_losses.append((mean[0]+alpha*std[0], (lr, mom, bs)))
          else:
            predicted_losses.append((regr.predict([[lr, mom, bs]])[0], (lr, mom, bs)))
      else:
        if withUncertainty:
          mean, std = regr.predict([[lr, mom]], return_std=True)
          predicted_losses.append((mean[0]+alpha*std[0], (lr, mom)))
        else:
          predicted_losses.append((regr.predict([[lr, mom]])[0], (lr, mom)))

  predicted_losses.sort(key=lambda x: x[0])

  prediction = get_prediction(df, predicted_losses[:5], in3D)

  return prediction, df

def predict_best_HPs(dataset_name, mode, in3D=False):
  client_data = get_client_data(dataset_name)
  samples_per_client = client_data[client_data['clientID']==1].shape[0]
  clientIDs = np.unique(client_data['clientID'].tolist())

  regressors = []

  df = pd.DataFrame()
  times = 0
  for client_id in clientIDs:
    tmp = client_data.loc[client_data.clientID == client_id]
    tmp = tmp[['client_lr', 'client_mom', 'batch_size', 'accuracy', 'loss']]
    df = pd.concat([df, tmp])

    X, y = tmp[['client_lr', 'client_mom']].values, tmp[['loss']].values
    if in3D:
      X, y = tmp[['client_lr', 'client_mom', 'batch_size']].values, tmp[['loss']].values

    regr = RandomForestRegressor(random_state=12)
    regr.fit(X, y.ravel())
    regressors.append(regr)

  srvr_time = 0
  lr_range, mom_range, bs_range = get_lr_mom_bs_range(dataset_name, df)

  predicted_losses = []
  for lr in lr_range:
    for mom in mom_range:
      if in3D:
        for bs in bs_range:
          predictions = [regr.predict([[lr, mom, bs]])[0] for regr in regressors]
          if mode == "APLM":
            predicted_losses.append((np.mean(predictions), (lr, mom, bs)))
          elif mode == "MPLM":
            predicted_losses.append((max(predictions), (lr, mom, bs)))
      else:
        predictions = [regr.predict([[lr, mom]])[0] for regr in regressors]
        if mode == "APLM":
          predicted_losses.append((np.mean(predictions), (lr, mom)))
        elif mode == "MPLM":
          predicted_losses.append((max(predictions), (lr, mom)))

  predicted_losses.sort(key=lambda x: x[0])

  prediction = get_prediction(df, predicted_losses[:5], in3D)

  return prediction, df

In [6]:
in3D = False
dataset_name = 'cifar10'
skew_type = 'qty'
skew = 0.1
nr_clients = 10

dataset_name = f'{DATA_PATH}/{dataset_name}_{skew_type}_skew_{skew}_{nr_clients}clients'

prediction, df = predict_best_HPs_SGM(dataset_name, in3D = in3D)
display(prediction)

prediction, df = predict_best_HPs_SGM(dataset_name, withUncertainty=True, in3D = in3D)
display(prediction)

prediction, df = predict_best_HPs(dataset_name, mode="MPLM", in3D = in3D)
display(prediction)

prediction, df = predict_best_HPs(dataset_name, mode="APLM", in3D = in3D)
display(prediction)


[(0.07753167080239703, (0.1, 0.9, 8)),
 (0.07768656874515756, (0.5, 0.6, 8)),
 (0.07793442028240241, (0.07, 0.9, 16)),
 (0.07797326922204899, (0.07, 0.95, 16)),
 (0.07833223316394704, (0.3, 0.6, 8))]

[(array([0.07784855]), (0.1, 0.95, 16)),
 (array([0.07800859]), (0.5, 0.3, 8)),
 (array([0.07802208]), (0.1, 0.9, 8)),
 (array([0.07814521]), (0.5, 0.6, 8)),
 (array([0.07842887]), (0.3, 0.6, 8))]

[(0.08954824662542653, (0.3, 0.9, 8)),
 (0.08970408323336214, (0.5, 0.3, 8)),
 (0.08978268160121072, (0.3, 0.6, 8)),
 (0.08981584535666821, (0.3, 0.3, 8)),
 (0.08982276789341415, (0.5, 0.9, 8))]

[(0.07748680618387681, (0.1, 0.9, 8)),
 (0.07766424154684241, (0.5, 0.6, 8)),
 (0.07798954604792159, (0.3, 0.6, 8)),
 (0.07818790482880679, (0.07, 0.9, 16)),
 (0.07842994364178664, (0.07, 0.95, 16))]

In [9]:
EXPORT_PATH = "results/flora_extended_data/SGM" 
if not os.path.exists(f"{EXPORT_PATH}/"):
    os.mkdir(f"{EXPORT_PATH}/")

for dataset_name in DATASETS:
    for skew_type in SKEWS.keys():
        results = []
        print(f"DATASET: {dataset_name}, SKEW_TYPE: {skew_type}")
        for nr_clients in NR_PARTIES:
            for skew in SKEWS[skew_type]:
                print(f"{nr_clients} clients, skew={skew}")
                fedavg_acc = get_federated_val_acc(dataset_name, skew, nr_clients, skew_type)
                res = get_federated_res(dataset_name, skew, nr_clients, skew_type)

                dataset_file_name = f'{DATA_PATH}/{dataset_name}_{skew_type}_skew_{skew}_{nr_clients}clients'
                
                prediction, df = predict_best_HPs_SGM(dataset_file_name, in3D=False)
                prediction3d, df = predict_best_HPs_SGM(dataset_file_name, in3D=True)

                # prediction, df = predict_best_HPs_SGM(dataset_file_name, withUncertainty=True, in3D=False)
                # prediction3d, df = predict_best_HPs_SGM(dataset_file_name, withUncertainty=True, in3D=True)
                
                # prediction, df = predict_best_HPs(dataset_file_name, mode="MPLM", in3D=False)
                # prediction3d, df = predict_best_HPs(dataset_file_name, mode="MPLM", in3D=True)

                # prediction, df = predict_best_HPs(dataset_file_name, mode="APLM", in3D=False)
                # prediction3d, df = predict_best_HPs(dataset_file_name, mode="APLM", in3D=True)

                derived_params = get_derived_params(prediction, dataset_file_name, fedavg_acc)
                derived_params3d = get_derived_params(prediction3d, dataset_file_name, fedavg_acc)
                
                results.append({
                        **{
                           "dataset": dataset_name,
                           "nr_clients": nr_clients,
                           "skew_type": skew_type,
                           "skew": skew,
                           "global_acc": fedavg_acc,
                           
                           "global_lr_mom_bs_acc": (res["server_lr"], res["server_momentum"],res["batch_size"],fedavg_acc),
                           
                           "derived_params": derived_params,
                           "derived_params3d": derived_params3d,
                           }
                    })

        file_name = (f"{EXPORT_PATH}/{dataset_name}_{skew_type}.json")

        with open(file_name, 'w') as outfile:
            json.dump(results, outfile)    


DATASET: mnist, SKEW_TYPE: feature
10 clients, skew=0.02
10 clients, skew=0.1
20 clients, skew=0.02
20 clients, skew=0.1
DATASET: mnist, SKEW_TYPE: label
10 clients, skew=0.1
10 clients, skew=1.0
10 clients, skew=5.0
20 clients, skew=0.1
20 clients, skew=1.0
20 clients, skew=5.0
DATASET: mnist, SKEW_TYPE: qty
10 clients, skew=0.1
10 clients, skew=0.4
10 clients, skew=1.0
10 clients, skew=2.0
20 clients, skew=0.1
20 clients, skew=0.4
20 clients, skew=1.0
20 clients, skew=2.0
DATASET: svhn_cropped, SKEW_TYPE: feature
10 clients, skew=0.02
10 clients, skew=0.1
20 clients, skew=0.02
20 clients, skew=0.1
DATASET: svhn_cropped, SKEW_TYPE: label
10 clients, skew=0.1
10 clients, skew=1.0
10 clients, skew=5.0
20 clients, skew=0.1
20 clients, skew=1.0
20 clients, skew=5.0
DATASET: svhn_cropped, SKEW_TYPE: qty
10 clients, skew=0.1
10 clients, skew=0.4
10 clients, skew=1.0
10 clients, skew=2.0
20 clients, skew=0.1
20 clients, skew=0.4
20 clients, skew=1.0
20 clients, skew=2.0
DATASET: cifar10, SKE